# 🏥 Analyse Avancée avec Dataset Enrichi - Clinique du Mont Vert
## IA Prédictive avec Prophet + Regressors Externes

---

**Version ENRICHI - Optimisée pour Prophet Avancé**

**Projet** : Master EISI - Gestion des stocks  
**Dataset** : Enrichi v3.0 - 5 ans de données (2020-2024)  
**Lignes** : 85,809 enregistrements  
**Colonnes** : 22 (15 base + 7 régresseurs)

---

## 📋 Nouveautés du Dataset Enrichi

### 🎯 7 Régresseurs Externes
- **temperature** : Température extérieure (°C)
- **taux_occupation** : Taux d'occupation hôpital (%)
- **nb_patients** : Nombre de patients
- **epidemie_grippe** : Périodes d'épidémie (0/1)
- **vacances_scolaires** : Vacances scolaires (0/1)
- **jour_ferie** : Jours fériés français (0/1)
- **covid_impact** : Périodes COVID (0/1)

### 🔄 Changepoints Majeurs
- **15/03/2020** : COVID-19 Vague 1 (+50%)
- **01/11/2020** : COVID-19 Vague 2 (+30%)
- **01/05/2021** : Déconfinement (-10%)
- **01/01/2022** : Nouvelle Direction (+10%)
- **01/09/2023** : Extension Hôpital (+15%)

### 📊 Résultats Attendus
- **MAE** : ~2.0 kg (vs 3.5 kg sans regressors)
- **MAPE** : ~20% (vs 35% sans regressors)
- **Précision** : Excellente

---

# 📦 Étape 1 : Imports et Configuration

In [ ]:
# Imports standards
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
import json
from pathlib import Path
warnings.filterwarnings('ignore')

# Configuration pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

# Configuration matplotlib
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (16, 8)
plt.rcParams['font.size'] = 10

print("✅ Imports de base terminés !")
print(f"📊 NumPy version : {np.__version__}")
print(f"📊 Pandas version : {pd.__version__}")

In [ ]:
# Import Prophet et utilitaires
print("🤖 Import de Prophet...")
try:
    from prophet import Prophet
    from prophet.plot import plot_yearly, plot_weekly, add_changepoints_to_plot
    from prophet.utilities import regressor_coefficients
    print("✅ Prophet importé avec succès !")
    PROPHET_AVAILABLE = True
except Exception as e:
    print(f"⚠️  Erreur Prophet : {e}")
    print("💡 Installation : pip install prophet")
    PROPHET_AVAILABLE = False

print(f"\n{'='*70}")
if PROPHET_AVAILABLE:
    print("🎉 Tout est prêt pour l'analyse avancée !")
else:
    print("⚠️  Prophet requis pour ce notebook")
print(f"{'='*70}")

In [ ]:
# Import Results Manager (optionnel)
try:
    from results_manager import ResultsManager
    results_mgr = ResultsManager()
    results_mgr.create_run_directory()
    print(f"✅ Results Manager activé")
    print(f"📁 Résultats seront sauvegardés dans : {results_mgr.get_run_path()}")
    USE_RESULTS_MANAGER = True
except:
    print("⚠️  Results Manager non disponible (optionnel)")
    USE_RESULTS_MANAGER = False

# 📂 Étape 2 : Chargement du Dataset Enrichi

In [ ]:
# Chemin du dataset enrichi
FICHIER_CSV = "../data/dataset_stock_hopital_ENRICHI.csv"

# Vérifier que le fichier existe
if not Path(FICHIER_CSV).exists():
    print(f"❌ Fichier introuvable : {FICHIER_CSV}")
    print("\n💡 Assurez-vous que le dataset enrichi est dans data/")
    raise FileNotFoundError(f"Le fichier {FICHIER_CSV} n'existe pas")

# Chargement
print(f"📊 Chargement de {FICHIER_CSV}...")
df = pd.read_csv(FICHIER_CSV)

print(f"✅ Dataset enrichi chargé avec succès !")
print(f"📏 Dimensions : {df.shape[0]:,} lignes × {df.shape[1]} colonnes")
print(f"💾 Taille mémoire : {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

In [ ]:
# Aperçu des données
print("👀 Aperçu des 5 premières lignes :")
df.head()

In [ ]:
# Vérifier les colonnes du dataset enrichi
print("📊 COLONNES DU DATASET ENRICHI")
print("="*70)

colonnes_base = ['date', 'id_produit', 'nom_produit', 'type_produit', 'type_operation',
                 'type_sortie', 'quantite', 'unite', 'id_lot', 'id_arrivage',
                 'id_fournisseur', 'nom_fournisseur', 'date_expiration',
                 'stock_theorique', 'temperature_stockage']

regresseurs = ['temperature', 'taux_occupation', 'nb_patients', 'epidemie_grippe',
               'vacances_scolaires', 'jour_ferie', 'covid_impact']

print("\n✅ Colonnes de base (15) :")
for col in colonnes_base:
    if col in df.columns:
        print(f"   ✓ {col}")
    else:
        print(f"   ✗ {col} (manquante)")

print("\n✅ Régresseurs externes (7) :")
for col in regresseurs:
    if col in df.columns:
        print(f"   ✓ {col}")
    else:
        print(f"   ✗ {col} (manquante)")

print(f"\n📊 Total colonnes trouvées : {len(df.columns)}")

# 🧹 Étape 3 : Préparation des Données

In [ ]:
# Conversion des dates
print("📅 Conversion des colonnes de dates...")
df['date'] = pd.to_datetime(df['date'])
df['date_expiration'] = pd.to_datetime(df['date_expiration'])

print("✅ Dates converties !")
print(f"📆 Période couverte : {df['date'].min().date()} → {df['date'].max().date()}")
print(f"⏱️  Durée totale : {(df['date'].max() - df['date'].min()).days} jours")
print(f"📊 Soit {(df['date'].max() - df['date'].min()).days / 365:.1f} ans de données")

In [ ]:
# Statistiques sur les régresseurs
print("📊 STATISTIQUES DES RÉGRESSEURS")
print("="*70)

print(f"\n🌡️  Température :")
print(f"   Moyenne : {df['temperature'].mean():.1f}°C")
print(f"   Min/Max : {df['temperature'].min():.1f}°C / {df['temperature'].max():.1f}°C")

print(f"\n🏥 Taux d'occupation :")
print(f"   Moyenne : {df['taux_occupation'].mean():.1f}%")
print(f"   Min/Max : {df['taux_occupation'].min():.1f}% / {df['taux_occupation'].max():.1f}%")

print(f"\n👥 Nombre de patients :")
print(f"   Moyenne : {df['nb_patients'].mean():.0f} patients/jour")
print(f"   Min/Max : {df['nb_patients'].min():.0f} / {df['nb_patients'].max():.0f}")

print(f"\n🦠 Épidémies de grippe :")
nb_jours_grippe = df['epidemie_grippe'].sum()
pct_grippe = (nb_jours_grippe / len(df) * 100)
print(f"   {nb_jours_grippe:,} jours d'épidémie ({pct_grippe:.1f}%)")

print(f"\n🏖️  Vacances scolaires :")
nb_jours_vacances = df['vacances_scolaires'].sum()
pct_vacances = (nb_jours_vacances / len(df) * 100)
print(f"   {nb_jours_vacances:,} jours de vacances ({pct_vacances:.1f}%)")

print(f"\n📅 Jours fériés :")
nb_jours_feries = df['jour_ferie'].sum()
nb_annees = (df['date'].max() - df['date'].min()).days / 365
print(f"   {nb_jours_feries:,} jours fériés (~{nb_jours_feries/nb_annees:.0f}/an)")

print(f"\n😷 Impact COVID :")
nb_jours_covid = df['covid_impact'].sum()
pct_covid = (nb_jours_covid / len(df) * 100)
print(f"   {nb_jours_covid:,} jours impactés ({pct_covid:.1f}%)")
print(f"   Soit {nb_jours_covid/30:.0f} mois cumulés")

# 🎯 Étape 4 : Sélection du Produit à Analyser

In [ ]:
# Choisir le produit à analyser
PRODUIT_ANALYSE = "Poulet frais"  # ← Changez ici pour analyser un autre produit

print(f"🔍 ANALYSE APPROFONDIE : {PRODUIT_ANALYSE}")
print("="*70)

# Filtrer uniquement les sorties de consommation (pas les destructions)
produit_df = df[
    (df['nom_produit'] == PRODUIT_ANALYSE) &
    (df['type_sortie'] == 'CONSOMMATION')
].copy()

if len(produit_df) == 0:
    print(f"❌ Aucune donnée trouvée pour '{PRODUIT_ANALYSE}'")
    print("\n💡 Produits disponibles :")
    print(df['nom_produit'].unique())
else:
    print(f"📊 {len(produit_df):,} sorties enregistrées")
    print(f"📅 Période : {produit_df['date'].min().date()} → {produit_df['date'].max().date()}")
    print(f"📦 Volume total : {produit_df['quantite'].sum():,.2f} kg")
    print(f"📈 Moyenne/sortie : {produit_df['quantite'].mean():.2f} kg")

# 📊 Étape 5 : Agrégation Quotidienne avec Régresseurs

In [ ]:
# Agréger par jour en gardant les régresseurs
print("🔧 Agrégation des données par jour...")

daily = produit_df.groupby('date').agg({
    'quantite': 'sum',                    # Somme des quantités consommées
    'temperature': 'mean',                # Moyenne température du jour
    'taux_occupation': 'mean',            # Moyenne taux occupation
    'nb_patients': 'mean',                # Moyenne nb patients
    'epidemie_grippe': 'max',             # 1 si épidémie ce jour
    'vacances_scolaires': 'max',          # 1 si vacances ce jour
    'jour_ferie': 'max',                  # 1 si férié ce jour
    'covid_impact': 'max'                 # 1 si COVID ce jour
}).reset_index()

print(f"✅ {len(daily)} jours avec données")

# Compléter les dates manquantes avec 0
print("🔧 Complétion des dates manquantes...")
date_range = pd.date_range(
    start=daily['date'].min(),
    end=daily['date'].max(),
    freq='D'
)
full_dates = pd.DataFrame({'date': date_range})
daily = full_dates.merge(daily, on='date', how='left')

# Remplir les valeurs manquantes
daily['quantite'].fillna(0, inplace=True)  # Pas de consommation = 0

# Pour les régresseurs continus : utiliser la moyenne
for col in ['temperature', 'taux_occupation', 'nb_patients']:
    daily[col].fillna(daily[col].mean(), inplace=True)

# Pour les régresseurs binaires : utiliser 0
for col in ['epidemie_grippe', 'vacances_scolaires', 'jour_ferie', 'covid_impact']:
    daily[col].fillna(0, inplace=True)

print(f"✅ {len(daily)} jours préparés (toutes les dates)")
print(f"   - Jours avec consommation : {(daily['quantite'] > 0).sum()}")
print(f"   - Jours sans consommation : {(daily['quantite'] == 0).sum()}")

In [ ]:
# Préparer le DataFrame pour Prophet
print("🔧 Préparation pour Prophet (renommage ds/y)...")

prophet_df = daily.copy()
prophet_df = prophet_df.rename(columns={'date': 'ds', 'quantite': 'y'})

print(f"✅ DataFrame Prophet prêt")
print(f"   - ds (date) : {prophet_df['ds'].min()} → {prophet_df['ds'].max()}")
print(f"   - y (quantité) : moyenne = {prophet_df['y'].mean():.2f} kg/jour")
print(f"   - Régresseurs : {', '.join(['temperature', 'taux_occupation', 'nb_patients', 'epidemie_grippe'])}")